<a href="https://colab.research.google.com/github/aritrasen87/LLM/blob/main/LLamaIndex_AutoMergingRetriever.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Required Installations

In [ ]:
! pip install llama-index llama-index-readers-web --q
! pip install llama-index-embeddings-huggingface --q
! pip install llama-index-llms-openai --q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 50.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.9/37.9 MB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 61.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16

### Loading web page as document

In [ ]:
from llama_index.readers.web import SimpleWebPageReader
from IPython.display import Markdown, display
import os

documents = SimpleWebPageReader(html_to_text=True).load_data(
    ["http://paulgraham.com/worked.html"]
)

### LLM & Embedding Model Initialization

In [ ]:
from llama_index.llms.openai import OpenAI
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_KEY')

llm = OpenAI(model="gpt-4o-mini-2024-07-18", temperature=0.0)

from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5", max_length=512)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/777 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

### Build HierarchicalNodeParser

In [ ]:
from llama_index.core.node_parser import HierarchicalNodeParser
from llama_index.core.node_parser import get_leaf_nodes

In [ ]:
chunk_sizes = [512, 256 ,128]

#node parser creation
node_parser = HierarchicalNodeParser.from_defaults(chunk_sizes=chunk_sizes)

nodes = node_parser.get_nodes_from_documents(documents)

leaf_nodes = get_leaf_nodes(nodes)

In [ ]:
len(leaf_nodes)

229

### Looking at leaf and parents nodes

- leaf nodes doesn't have any children

In [ ]:
print(leaf_nodes[50].text)

In everyday
life it would be distracting to notice every leaf on every bush. But when you
have to paint something, you have to look more closely, and when you do
there's a lot to see. You can still be noticing new things after days of
trying to paint something people usually take for granted, just as you can
after days of trying to write an essay about something people usually take for
granted.  
  
This is not the only way to paint. I'm not 100% sure it's even a good way to
paint.


In [ ]:
nodes_by_id = {node.node_id: node for node in nodes}

parent_node = nodes_by_id[leaf_nodes[50].parent_node.node_id]
print(parent_node.text)

[4]  
  
I liked painting still lives because I was curious about what I was seeing. In
everyday life, we aren't consciously aware of much we're seeing. Most visual
perception is handled by low-level processes that merely tell your brain
"that's a water droplet" without telling you details like where the lightest
and darkest points are, or "that's a bush" without telling you the shape and
position of every leaf. This is a feature of brains, not a bug. In everyday
life it would be distracting to notice every leaf on every bush. But when you
have to paint something, you have to look more closely, and when you do
there's a lot to see. You can still be noticing new things after days of
trying to paint something people usually take for granted, just as you can
after days of trying to write an essay about something people usually take for
granted.  
  
This is not the only way to paint. I'm not 100% sure it's even a good way to
paint. But it seemed a good enough bet to be worth trying.  
  


### Steps

1. Service Context creation
2. Storage Context creation
3. VectoreStoreIndex creation



In [ ]:
from llama_index.core import ServiceContext, VectorStoreIndex, StorageContext

auto_merging_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model=embed_model,
    node_parser=node_parser,
)

<ipython-input-12-5ac1161b2e96>:3: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  auto_merging_context = ServiceContext.from_defaults(



*   Parent nodes are stored in DocStore (StorageContext)
*   Index is created using leaf nodes and we only embed the leaf nodes (VectorStoreIndex)
*   Relationships are maintained between parent and leaf and during retrieval merging happens

In [ ]:
### DocStore creation using all nodes

storage_context = StorageContext.from_defaults()
storage_context.docstore.add_documents(nodes)

In [ ]:
### VectoreStore Index creation

automerging_index = VectorStoreIndex(leaf_nodes,
                                     storage_context=storage_context,
                                     service_context=auto_merging_context)

In [ ]:
re_rank = True
similarity_top_k = 12
rerank_top_n = 4

In [ ]:
from llama_index.core.retrievers import AutoMergingRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.indices.postprocessor import SentenceTransformerRerank

In [ ]:
base_retriever = automerging_index.as_retriever(similarity_top_k=similarity_top_k)
retriever = AutoMergingRetriever(base_retriever, automerging_index.storage_context, verbose=True)

if re_rank:
    rerank = SentenceTransformerRerank(top_n=rerank_top_n, model="BAAI/bge-reranker-base")
    auto_merging_engine = RetrieverQueryEngine.from_args(retriever,llm=llm,node_postprocessors=[rerank])
else:
    auto_merging_engine = RetrieverQueryEngine.from_args(retriever,llm=llm)

config.json:   0%|          | 0.00/799 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

In [ ]:
def auto_merging_retriever(auto_merging_engine,query):

    merging_response = auto_merging_engine.query(query)

    context = []

    for node in merging_response.source_nodes:
        context.append(node.dict()['node']['text'])

    response = merging_response.response

    return response , context

In [ ]:
query = "What did the author do growing up?"

In [ ]:
response , context = auto_merging_retriever(auto_merging_engine,query)

> Merging 1 nodes into parent node.
> Parent node id: 84fed834-d917-452b-8776-5e9dcbb5b5d2.
> Parent node text: I kept writing essays through 2020, but I also started to think
about other things I could work o...



In [ ]:
response

'Growing up, the author worked on writing and programming outside of school. They focused on writing short stories, although they considered their early stories to be lacking in plot and depth, primarily featuring characters with strong feelings.'

In [ ]:
context

['Even then it took me several years to\nunderstand the implications. It meant there would be a whole new generation of\n[_essays_](essay.html). [11]  \n  \nIn the print era, the channel for publishing essays had been vanishingly\nsmall. Except for a few officially anointed thinkers who went to the right\nparties in New York, the only people allowed to publish essays were\nspecialists writing about their specialties. There were so many essays that\nhad never been written, because there had been no way to publish them.',
 "I remember that I answered the\nessay question by writing about Cezanne, and that I cranked up the\nintellectual level as high as I could to make the most of my limited\nvocabulary. [2]  \n  \nI'm only up to age 25 and already there are such conspicuous patterns. Here I\nwas, yet again about to attend some august institution in the hopes of\nlearning about some prestigious subject, and yet again about to be\ndisappointed.",
 "[What I Worked On](https://s.turbifycdn.co